In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def make_cartesian_plane(ax):
    ax.spines["top"].set_color("none")
    ax.spines["bottom"].set_position("zero")
    ax.spines["left"].set_position("zero")
    ax.spines["right"].set_color("none")


def remove_spines(ax):
    ax.spines[["right", "top"]].set_visible(False)


In [42]:
def haar(ts):
    ys = np.ones_like(ts)
    ys[(ts >= 0) & (ts < 0.5)] = 1
    ys[(ts >= 0.5) & (ts < 1)] = -1
    ys[ts < 0] = 0
    ys[ts >= 1] = 0
    
    return ys

remove_spines(plt.gca())
xs = np.linspace(0, 1, 1024)
plt.plot(xs, haar(xs));

In [43]:
def chirp(ts):
    ys = np.cos(50*ts**2)

    return ys

remove_spines(plt.gca())
xs = np.linspace(0, 1, 1024)
plt.plot(xs, chirp(xs));

In [47]:
plt.figure(figsize=(12,12))

for i in range(4):
    plt.subplot(2, 2, i + 1)
    plt.plot(xs, chirp(xs));
    plt.plot(xs, haar(xs - 0.25*i));
    remove_spines(plt.gca())


# CWT

In [27]:
n_steps = 1024
ts = np.linspace(0, 1, n_steps)
ys = chirp(ts)
wavelet = haar(ts)
h = ts[1] - ts[0]

plt.figure()
result = []
for b in range(1024):
    # this only works because we start and end at the same time
    # y_snip = ys[b:]
    # wavelet_snip = wavelet[:n_steps - b]
    # result.append(np.dot(y_snip, wavelet_snip))
    # if b % 512==0:
    #     plt.plot(ts - b * h, haar(ts - b * h))
plt.plot(ts, result)
    # need to translate the Haar-wavelet and perform a sort of dot product. More like auto-correlation
        
        

In [14]:
import numpy as np

def haar_dwt(signal):
    """Performs a single-level Haar Discrete Wavelet Transform (DWT)."""
    N = len(signal)
    if N % 2 != 0:
        raise ValueError("Signal length must be even for this simple implementation.")

    approx = np.zeros(N // 2)  # Approximation coefficients
    detail = np.zeros(N // 2)  # Detail coefficients

    for i in range(0, N, 2):
        approx[i // 2] = (signal[i] + signal[i+1]) / np.sqrt(2)  # Low-pass filter
        detail[i // 2] = (signal[i] - signal[i+1]) / np.sqrt(2)  # High-pass filter

    return approx, detail

def haar_idwt(approx, detail):
    """Performs a single-level inverse Haar Discrete Wavelet Transform (IDWT)."""
    N = len(approx) * 2
    signal = np.zeros(N)

    for i in range(len(approx)):
        signal[2*i] = (approx[i] + detail[i]) / np.sqrt(2)  # Reconstruct even index
        signal[2*i+1] = (approx[i] - detail[i]) / np.sqrt(2)  # Reconstruct odd index

    return signal

# Example signal (length must be even)
signal = np.array([4, 6, 8, 8, 6, 4, 2, 1])

# Compute Haar DWT
cA, cD = haar_dwt(signal)

# Reconstruct the signal
reconstructed_signal = haar_idwt(cA, cD)

# Print results
print("Original Signal: ", signal)
print("Approximation Coefficients: ", cA)
print("Detail Coefficients: ", cD)
print("Reconstructed Signal: ", reconstructed_signal)


In [15]:
import numpy as np
import matplotlib.pyplot as plt

def haar_dwt(signal):
    """Performs a single-level Haar Discrete Wavelet Transform (DWT)."""
    N = len(signal)
    if N % 2 != 0:
        raise ValueError("Signal length must be even.")

    approx = np.zeros(N // 2)
    detail = np.zeros(N // 2)

    for i in range(0, N, 2):
        approx[i // 2] = (signal[i] + signal[i+1]) / np.sqrt(2)  # Low-pass filter
        detail[i // 2] = (signal[i] - signal[i+1]) / np.sqrt(2)  # High-pass filter

    return approx, detail

def multi_level_dwt(signal, max_levels):
    """Computes multi-level Haar DWT."""
    approximations, details = [], []
    current_signal = signal

    for _ in range(max_levels):
        if len(current_signal) < 2:
            break  # Stop when the signal is too short

        approx, detail = haar_dwt(current_signal)
        approximations.append(approx)
        details.append(detail)
        current_signal = approx  # Apply DWT to the approximation part

    return approximations, details

def plot_wavelet_spectrogram(details):
    """Plots a wavelet spectrogram using the detail coefficients."""
    plt.figure(figsize=(8, 6))
    
    # Convert detail coefficients into a matrix for visualization
    spectrogram = np.array([np.abs(d) for d in details])  # Take absolute values for magnitude
    spectrogram = np.flipud(spectrogram)  # Flip to match spectrogram orientation

    plt.imshow(spectrogram, aspect='auto', cmap='plasma', interpolation='nearest')
    plt.colorbar(label="Magnitude")
    plt.xlabel("Time (samples)")
    plt.ylabel("Wavelet Scale (Higher = Lower Frequency)")
    plt.title("Wavelet Transform Spectrogram (Haar)")
    plt.show()

# Example signal (must be a power of 2 for simplicity)
signal = np.array([4, 6, 8, 8, 6, 4, 2, 1])

# Perform multi-level DWT
num_levels = 3
approximations, details = multi_level_dwt(signal, num_levels)

# Plot the wavelet spectrogram
# plot_wavelet_spectrogram(details)


In [4]:
from scipy import signal
import matplotlib.pyplot as plt

t = np.linspace(-1, 1, 200, endpoint=False)
sig  = np.cos(2 * np.pi * 7 * t) + signal.gausspulse(t - 0.4, fc=2)
widths = np.arange(1, 31)

In [5]:
plt.figure()
plt.plot(t, sig)

In [6]:
plt.figure()

cwtmatr = signal.cwt(sig, signal.ricker, widths)
plt.imshow(cwtmatr, extent=[-1, 1, 1, 31], cmap='PRGn', aspect='auto', vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())
plt.show()